## Redes Neurais Artificiais 2020.1

**Disciplina**: Redes Neurais Artificiais 2020.1  
**Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
**Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta terceira parte do Projeto Prático 3 diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Testando Redes Neurais sem os Atributos Categórios

1. Abra a base de dados em questão
2. Elimine todas as colunas relativas aos atributos categóricos
3. Armazene o atributo alvo em uma variável y e os atributos preditores em uma variável X
4. Efetue uma partição holdout 70/30 com o sklearn, distribuindo os exemplos de maneira aleatória
5. Efetue o escalonamento dos atributos

### Escalonando os atributos

O treinamento de uma rede neural artificial é mais eficiente quando os valores que lhes são fornecidos como entrada são pequenos, pois isto favorece a convergência. Isto é feito escalonando-se todos os atributos para o intervalo [0,1], mas precisa ser feito de maneira cautelosa, para que informações do conjunto de teste não sejam fornecidas no treinamento.

Há duas estratégias para tal escalonamento: normalização e padronização. Ambas possuem características particulares, vantagens e limitações, como é possível ver aqui: https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/


No nosso caso, vamos usar a padronização. Assim, com os atributos preditores do treinamento, isto é, X_train, deve-se subtrair a média e dividir pelo desvio padrão:

X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

Em seguida, o mesmo deve ser feito com os atributos preditores do conjunto de testes, mas com padronização relativa ao conjunto de treinamento:

X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

Se todo o conjunto X for utilizado na padronização, a rede neural receberá informações do conjunto de teste por meio da média e variância utilizada para preparar os dados de treinamento, o que não é desejável.


### Continuando

5. Treine uma rede neural multilayer perceptron para este problema com uma única camada e dez neurônios  
    5.1 Utilize a função de ativação ReLU  
    5.2 Utilize o solver Adam    
    5.3 Imprima o passo a passo do treinamento    
    5.4 Utilize o número máximo de épocas igual a 300  
6. Com o modelo em questão, após o treinamento, apresente:  
    6.1 Matriz de confusão para o conjunto de teste  
    6.2 Acurácia  
    6.3 F-Score  
    6.4 Precisão  
    6.5 Revocação  
7. Repita o treinamento da mesma rede anterior sem imprimir o passo a passo (verbose False) por 100 vezes  
    7.1 Cada uma destas repetições deve ser feita com uma nova partição Holdout  
    7.2 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
8. Repita por 100 vezes o treinamento desta mesma rede, mas utilizando o otimizador SGD  
    8.1 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino  
9. Houve influência da escolha do otimizador no desempenho da rede?

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving covtype.csv to covtype.csv


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

#### 1. Abra a base de dados em questão

In [ ]:
data = pd.read_csv('covtype.csv')
data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3


#### 2. Elimine todas as colunas relativas aos atributos categóricos

In [ ]:
#Armazenando nome de colunas a serem eliminadas em uma lista
columns_out = []

#Atributo área selvagem
for i in range(1, 5):
    columns_out.append('Wilderness_Area' + str(i))
    
#Atributo tipo de solo
for i in range(1, 41):
    columns_out.append('Soil_Type' + str(i))
    
data = data.drop(columns=columns_out)
data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,5
1,2590,56,2,212,-6,390,220,235,151,6225,5
2,2804,139,9,268,65,3180,234,238,135,6121,2
3,2785,155,18,242,118,3090,238,238,122,6211,2
4,2595,45,2,153,-1,391,220,234,150,6172,5
...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,3
581008,2391,152,19,67,12,95,240,237,119,845,3
581009,2386,159,17,60,7,90,236,241,130,854,3
581010,2384,170,15,60,5,90,230,245,143,864,3


#### 3. Armazene o atributo alvo em uma variável y e os atributos preditores em uma variável X

In [ ]:
#Dataframe apenas com os atributos preditores
data_atr = data.drop(columns=['Cover_Type'])

In [ ]:
X = np.array(data_atr[0:])

y = data['Cover_Type'].to_numpy()

#### 4. Efetue uma partição holdout 70/30 com o sklearn, distribuindo os exemplos de maneira aleatória

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#### 5. Efetue o escalonamento dos atributos

In [39]:
X_train_std = (X_train - np.mean(X_train))/np.std(X_train)

X_test_std = (X_test - np.mean(X_train))/np.std(X_train)

#### 6. Treine uma rede neural multilayer perceptron para este problema com uma única camada e dez neurônios  
    6.1 Utilize a função de ativação ReLU  
    6.2 Utilize o solver Adam    
    6.3 Imprima o passo a passo do treinamento    
    6.4 Utilize o número máximo de épocas igual a 300

In [40]:
clf = MLPClassifier(hidden_layer_sizes=(10,), max_iter=300, verbose=True).fit(X_train_std, y_train)

Iteration 1, loss = 1.06573825
Iteration 2, loss = 0.82971357
Iteration 3, loss = 0.77386106
Iteration 4, loss = 0.75227988
Iteration 5, loss = 0.74199029
Iteration 6, loss = 0.73589129
Iteration 7, loss = 0.73175981
Iteration 8, loss = 0.72813510
Iteration 9, loss = 0.72523633
Iteration 10, loss = 0.72042069
Iteration 11, loss = 0.71730955
Iteration 12, loss = 0.71569959
Iteration 13, loss = 0.71416829
Iteration 14, loss = 0.71274641
Iteration 15, loss = 0.71153703
Iteration 16, loss = 0.71003048
Iteration 17, loss = 0.70860642
Iteration 18, loss = 0.70743166
Iteration 19, loss = 0.70621060
Iteration 20, loss = 0.70524776
Iteration 21, loss = 0.70409852
Iteration 22, loss = 0.70328411
Iteration 23, loss = 0.70242568
Iteration 24, loss = 0.70162388
Iteration 25, loss = 0.70089664
Iteration 26, loss = 0.70011549
Iteration 27, loss = 0.69929698
Iteration 28, loss = 0.69853171
Iteration 29, loss = 0.69796751
Iteration 30, loss = 0.69741572
Iteration 31, loss = 0.69665831
Iteration 32, los

#### 7. Com o modelo em questão, após o treinamento, apresente:  
    7.1 Matriz de confusão para o conjunto de teste  
    7.2 Acurácia  
    7.3 F-Score  
    7.4 Precisão  
    7.5 Revocação  

In [41]:
y_pred = clf.predict(X_test_std)
print(confusion_matrix(y_test, y_pred))

print("")
print("Acurácia: " + str(np.round(clf.score(X_test_std, y_test)*100, 4)) + "%")

precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
print("Precisão:",np.round(precision, 4))

recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
print("Revocação:",np.round(recall, 4))

fscore = 2 * (precision * recall) / (precision + recall)
print("F-Score:",np.round(fscore, 4))

[[46086 16495     1     0     0    20   878]
 [17700 65692  1096     0     0   547    15]
 [    0  1923  7952   102     0   724     0]
 [    0     6   349   421     0    44     0]
 [    1  2763    39     0     0    21     0]
 [    0  1377  2594    41     0  1277     0]
 [ 3741    32     0     0     0     0  2367]]

Acurácia: 71.0225%
Precisão: 0.696
Revocação: 0.7102
F-Score: 0.703


#### 8. Repita o treinamento da mesma rede anterior sem imprimir o passo a passo (verbose False) por 100 vezes  
    8.1 Cada uma destas repetições deve ser feita com uma nova partição Holdout  
    8.2 Apresente a média e o desvio padrão da acurácia e do F-Score para o conjunto de treino 

In [42]:
def training(iterations, solver, max_epochs):
  #numpy array para armzanear as acurácias
  acurr_arr = np.array([])

  #numpy array para armzanear os F-scores
  fscore_arr = np.array([])

  for i in range(10):
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

      #Escalonamento
      X_train_std = (X_train - np.mean(X_train))/np.std(X_train)
      X_test_std = (X_test - np.mean(X_train))/np.std(X_train)
      
      clf = MLPClassifier(hidden_layer_sizes=(10,), solver=solver, max_iter=max_epochs, verbose=False).fit(X_train_std, y_train)
      y_pred = clf.predict(X_test_std)
      
      acurr_arr = np.append(acurr_arr, clf.score(X_test_std, y_test))
      fscore_arr = np.append(fscore_arr, f1_score(y_test, y_pred, average='weighted'))
      
  print("Acurácia")
  print("Média:" + str(np.round(np.mean(acurr_arr), 4)))
  print("Desvio padrão:" + str(np.round(np.std(acurr_arr), 4)))
  print("")
  print("F-Score")
  print("Média:" + str(np.round(np.mean(fscore_arr), 4)))
  print("Desvio padrão:" + str(np.round(np.std(fscore_arr), 4)))

In [44]:
training(10, 'adam', 300)

Acurácia
Média:0.7123
Desvio padrão:0.0047

F-Score
Média:0.6991
Desvio padrão:0.0053


#### 9. Repita por 100 vezes o treinamento desta mesma rede, mas utilizando o otimizador SGD  

In [46]:
training(10, 'sgd', 300)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Acurácia
Média:0.7034
Desvio padrão:0.0034

F-Score
Média:0.6867
Desvio padrão:0.0038


#### 10. Houve influência da escolha do otimizador no desempenho da rede?

O solver adam parece ter uma ligeira vantagem, visto que com o solver sgd houve uma maior dificuldade na convergência.

## Discussão

Nos passos anteriores, você avaliou o desempenho de uma única rede neural que contém os seguintes parâmetros: uma única camada oculta com 10 neurônios e função de ativação ReLU. O otimizador utilizado, quer seja SGD ou ADAM, trata-se do algoritmo para aproximar o gradiente do erro. Neste sentido, a escolha do otimizador é um hiperparâmetro, pois diz respeito a como a rede neural definida previamente atuará "em tempo de execução"  durante o processo de treinamento. Também são hiperparâmetros a quantidade de épocas, a taxa de aprendizado inicial, dentre outros.

Cabe alientar também que você efetuou o treinamento desta rede por 100 vezes e apresentou os resultados em termos de média +- desvio padrão. Lembre-se que em uma rede neural há a inicialização aleatória de pesos e, em consequência, o desempenho delas está sujeito à uma flutuação estocástica. A execução destas múltiplas vezes faz com que eliminemos algum viés introduzido por uma boa ou má "sorte" na escolha de pesos no caso de uma única execução.

Você também aprendeu uma estratégia para escalonar os atributos para uma melhor convergência da rede. Utilize-a em todos os treinamentos e testes propostos a seguir.

## Propondo Novas Arquiteturas

Variando  os parâmetros (uma ou duas camadas ocultas, com diferente números de neurônios em cada uma delas e a função de ativação) e o hiperparâmetros solver (Adam ou SGD) e o número de épocas (100,150 e 200), atenda ao que se pede:

1. Proponha 10 arquiteturas distintas de RNAs para o problema em questão, à sua escolha
2. Avalie cada uma das arquiteturas perante todos os hiperparâmetros apresentados por 100 vezes
3. Como resultado da avaliação, apresente:  
    3.1 Top-3 melhores redes no tocante à F-Score e Acurácia  
    3.2 Repetição em que houve o melhor desempenho de cada uma dessas redes: ilustre tp, tf, fp e fn  

#### 1. Proponha 10 arquiteturas distintas de RNAs para o problema em questão, à sua escolha.

*   Arquitetura 1 - 1 camada oculta, 1 neurônio, função ReLU
*   Arquitetura 2 - 1 camada oculta, 5 neurônios, função ReLU
*   Arquitetura 3 - 2 camada oculta, 10 neurônios, 9 neurônios, função ReLU
*   Arquitetura 4 - 2 camada oculta, 10 neurônios, 1 neurônio, função ReLU
*   Arquitetura 5 - 2 camada oculta, 10 neurônios, 5 neurônios, função ReLU
*   Arquitetura 6 - 2 camada oculta, 1 neurônio, 10 neurônios, função ReLU
*   Arquitetura 7 - 2 camada oculta, 5 neurônios, 10 neurônios, função ReLU
*   Arquitetura 8 - 1 camada oculta, 10 neurônios, função tangenta hiperbólica
*   Arquitetura 9 - 1 camada oculta, 10 neurônios, função logistica
*   Arquitetura 10 - 1 camada oculta, 10 neurônios, função identidade


## Estimando o número de neurônios

Um dos problemas de pesquisa com redes neurais artificiais consiste na determinação do número de neurônios em sua arquitetura. Embora não seja possível definir a priori qual rede neural é adequada para um problema, pois isto só é possível mediante uma busca exaustiva, há regras na literatura que sugerem o número de neurônios escondidos, tal como a regra da Pirâmide Geométrica, dada a seguir:

$$N_h = \alpha \cdot \sqrt{N_i \cdot N_o},$$

em que $N_h$ é o número de neurônios ocultos (a serem distribuídos em uma ou duas camadas ocultas), $N_i$ é o número de neurônios na camada de entrada e $N_o$ é o número de neurônios na camada de saída. 

1. Consulte a documentação da classe MLPClassifier (disponível em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) e obtenha os valores de $N_i$ e $N_h$.
2. Teste os valores de $\alpha$ como sendo iguais a $0.5$, $2$ e $3$.
3. Proponha pelo menos 30 redes neurais segundo a regra da pirâmide geométrica e teste-as nos mesmos termos estabelecidos anterioremente  (solver, épocas, etc.)  
    3.1 Apresente as top-3 melhores redes no tocante à F-Score e Acurácia  

## Testando as Redes Neurais com Atributos Categóricos

1. Considere as 6 redes neurais obtidas nos dois top-3 anteriores (arquiteturas próprias e regra da pirâmide geométrica)
2. Com todos os atributos preditores da base de dados original, incluindo os categóricos, treine e teste estas mesmas redes por 100 repetições  
    2.1 Considere o melhor otimizador para cada uma delas  
    2.2 Faça uso de 200 épocas para treinamento  
    2.2 Apresente os resultados de acurácia e F-Score em termos da média +- dp para cada arquitetura
3. Apresente o gráfico boxplot para o F-Score das 6 arquiteturas perante as 100 repetições

## Considerações Parciais

1. É possível identificar uma rede com desempenho superior às demais?
2. Qual estratégia mostrou-se mais producente para a obtenção de boas arquiteturas (Estratégia Própria ou Pirâmide Geométrica)? Por quê?
3. Considerar os atributos categóricos trouxe melhorias? Justifique.
4. Um número maior de épocas trouxe melhorias?
5. Qual a maior dificuldade de resolução do problema proposto perante as RNAs?